In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

def animate_lattice(dataset, fps=8, cmap="coolwarm"):
    fig, ax = plt.subplots(figsize=(5, 5))
    ax.set_axis_off()
    # dataset contains RGB arrays from to_two_color; recover spin lattice from red channel
    # red channel == 255 means spin -1, red == 0 means spin +1
    frames = []
    for frame in dataset:
        spins = np.where(frame[0] == 255, -1, 1)
        frames.append(spins)

    im = ax.imshow(frames[0], cmap=cmap, vmin=-1, vmax=1, interpolation="nearest")
    ax.set_title(f"Frame 0 / {len(frames)}")

    def update(i):
        im.set_data(frames[i])
        ax.set_title(f"Frame {i} / {len(frames)}")
        return [im]

    anim = animation.FuncAnimation(
        fig, update, frames=len(frames), interval=1000 // fps, blit=True
    )
    plt.close(fig)
    return HTML(anim.to_jshtml())

### numpy

In [3]:
import numpy as np

In [4]:
def to_two_color(lattice):
    blue = np.ones(lattice.shape, dtype=int) * 255 
    red = np.zeros(lattice.shape, dtype=int)
    red[lattice < 0] = 255 
    green = red 
    return np.array([red, green, blue])

def get_dH(lattice, trial_location):
    """ H = - Sum_<ij>(s_i s_j) """
    i, j = trial_location
    height, width = lattice.shape
    H, Hflip = 0, 0
    for di, dj in ((-1, 0), (1, 0), (0, -1), (0, 1)):
        ii = (i + di) % height
        jj = (j + dj) % width
        H -= lattice[ii, jj] * lattice[i, j]
        Hflip += lattice[ii, jj] * lattice[i, j]
    return Hflip - H

def standard_approach(T_over_Tc, width, height, N=60):
    # Randomly initialize the spins to either +1 or -1
    lattice = 2 * np.random.randint(2, size=(height, width)) - 1
    Tc = 2.269
    T = T_over_Tc * Tc
    snapshots = []
    for snapshot in range(N):
        snapshots.append(to_two_color(lattice))
        print('{:2.0%} complete. Net magnetization: {:3.0%}'
              .format(snapshot / N,
                      abs(lattice.sum()) / lattice.size),
              end='\r')
        for step in range(5):
            # Walk through the array flipping atoms.
            for i in range(height):
                for j in range(width):
                    dH = get_dH(lattice, (i, j))
                    if dH < 0:  # lower energy: flip for sure
                        lattice[i, j] = -lattice[i, j]
                    else:  # Higher energy: flip sometimes
                        probability = np.exp(-dH / T)
                        if np.random.rand() < probability:
                            lattice[i, j] = -lattice[i, j]
    return snapshots

In [5]:
snapshots = standard_approach(T_over_Tc=1.25, width=50, height=50, N=100)
animate_lattice(snapshots, fps=8)

### Numba JIT

In [ ]:
from numba import jit

@jit(nopython=True)

### PyMC

In [2]:
import pytensor
import pytensor.tensor as pt
import pymc as pm
from pytensor.compile.ops import as_op

In [3]:
pytensor.config.gcc__cxxflags = "-Wno-c++11-narrowing"

def get_H(spins):
    H = - (
        pt.roll(spins, 1, axis=1) * spins +
        pt.roll(spins, 1, axis=0) * spins +
        pt.roll(spins, -1, axis=1) * spins +
        pt.roll(spins, -1, axis=0) * spins
    )
    return H

def to_spins(lattice):
    return 2 * lattice - 1

def pymc_approach(T_over_Tc, width, height, N=100):
    Tc = 2.269
    T = T_over_Tc * Tc
    shape = (height, width)
    x0 = np.random.randint(2, size=shape)
    with pm.Model() as model:
        x = pm.Bernoulli('x', p=0.5, shape=shape, initval=x0)
        magnetization = pm.Potential(
            'm',
            -get_H(to_spins(x)) / T
        )
        scaling = .0006
        mul = int(height * width * 1.75)
        step = pm.BinaryMetropolis([x], scaling=scaling)
        trace = pm.sample(N * mul * 5, step=step, chains=1, tune=0)
    dataset = [to_two_color(2 * trace.posterior['x'].values[0, i] - 1) for i in range(0, N * mul * 5, mul * 5)]
    lattice = 2 * trace.posterior['x'].values[0, -1] - 1
    print('Finished. Net magnetization: {:3.0%}'
              .format(abs(lattice.sum()) / lattice.size))
    return dataset

In [ ]:
snapshots = pymc_approach(T_over_Tc=1.25, width=50, height=50, N=80)

NameError: name 'np' is not defined

In [ ]:
animate_lattice(snapshots, fps=8)